In [1]:
import io
import base64
import pandas as pd
import numpy as np
from helper_fcts import preprocess, get_poi_mas, rps
from keras import backend as K
from sklearn.preprocessing import scale, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from IPython.display import SVG, HTML
from operator import itemgetter
from scipy.stats import poisson
from mpl_toolkits.mplot3d import Axes3D
from sklearn.externals import joblib
from keras.models import load_model
from xgboost import XGBClassifier, XGBRegressor

from main import modelling

C:\Users\Konny\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df1 = pd.concat([modelling(pd.read_csv("data/E" + str(i) + ".csv"), 20) for i in range(8,19)])
#df2 = pd.concat([modelling(pd.read_csv("data/F" + str(i) + ".csv"), 20) for i in range(8,19)])
#df3 = pd.concat([modelling(pd.read_csv("data/D" + str(i) + ".csv"), 18) for i in range(8,19)])
#df4 = pd.concat([modelling(pd.read_csv("data/I" + str(i) + ".csv"), 20) for i in range(8,19)])
#df5 = pd.concat([modelling(pd.read_csv("data/SP" + str(i) + ".csv"), 20) for i in range(8,19)])

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


In [936]:
df1["country"] = "E"
df2["country"] = "F"
df3["country"] = "G"
df4["country"] = "I"
df5["country"] = "S"

In [942]:
df = pd.concat([df1, df2, df3, df4, df5])

In [944]:
df = pd.concat([df, pd.get_dummies(df.country)], axis=1)

### CLF

In [3]:
# columns_mit_fe  = ['H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']
# data2 = df5[df5["round"] > 10].reset_index(drop=True)
# data = data2[columns_mit_fe]
# data.dropna(inplace=True)

In [865]:
columns_mit_fe  = ['Home', 'Draw', 'Away', 'season', 'H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,3:]
    X_test = df[df["season"] == season].iloc[:,3:]
    y_train = df[df["season"] < season].iloc[:,:3]
    y_test = df[df["season"] == season].iloc[:,:3]
    return X_train, X_test, y_train, y_test

def loss_rps(y_true, y_pred):
    prob_h = y_pred[:, 0]
    prob_d = y_pred[:, 1]
    home = y_true[:, 0]
    draw = y_true[:, 1]

    step1 = prob_h - home
    step2 = prob_d - draw
    summe = step1 + step2
    return (step1 ** 2 + summe ** 2) / 2


data2 = df5[df5["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train, y_test = get_season(data, 18)

del X_train["season"]
del X_test["season"]

scaler = joblib.load("scaler_spa.save")

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [946]:
columns_mit_fe  = ['Home', 'Draw', 'Away', 'season', 'E', 'F', 'G', 'I', 'S', 'H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,3:]
    X_test = df[df["season"] == season].iloc[:,3:]
    y_train = df[df["season"] < season].iloc[:,:3]
    y_test = df[df["season"] == season].iloc[:,:3]
    return X_train, X_test, y_train, y_test

def loss_rps(y_true, y_pred):
    prob_h = y_pred[:, 0]
    prob_d = y_pred[:, 1]
    home = y_true[:, 0]
    draw = y_true[:, 1]

    step1 = prob_h - home
    step2 = prob_d - draw
    summe = step1 + step2
    return (step1 ** 2 + summe ** 2) / 2


data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train, y_test = get_season(data, 18)

del X_train["season"]
del X_test["season"]

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [947]:
K.clear_session()

model = Sequential()
model.add(Dense(128, kernel_initializer="he_normal", input_shape=(24,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(3, kernel_initializer="he_normal"))
model.add(Activation('softmax'))

model.compile(loss=loss_rps, optimizer='adam', metrics=['accuracy'])

In [948]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Train on 13343 samples, validate on 1332 samples
Epoch 1/100
13343/13343 [==============================] - 1s 106us/step - loss: 0.2647 - acc: 0.4298 - val_loss: 0.2103 - val_acc: 0.5278
Epoch 2/100
13343/13343 [==============================] - 1s 71us/step - loss: 0.2237 - acc: 0.4669 - val_loss: 0.1934 - val_acc: 0.5420
Epoch 3/100
13343/13343 [==============================] - 1s 70us/step - loss: 0.2068 - acc: 0.4983 - val_loss: 0.1928 - val_acc: 0.5420
Epoch 4/100
13343/13343 [==============================] - 1s 73us/step - loss: 0.2030 - acc: 0.5185 - val_loss: 0.1923 - val_acc: 0.5420
Epoch 5/100
13343/13343 [==============================] - 1s 75us/step - loss: 0.2007 - acc: 0.5251 - val_loss: 0.1916 - val_acc: 0.5458
Epoch 6/100
13343/13343 [==============================] - 1s 74us/step - loss: 0.1998 - acc: 0.5269 - val_loss: 0.1919 - val_acc: 0.5450
Epoch 7/100
13343/13343 [==============================] - 1s 74us/step - loss: 0.2000 - acc: 0.5316 - val_loss: 0.1921 - 

In [949]:
print("--------------------------------------------")
print(f"Train Dataset:\n RPS: {np.round(model.evaluate(X_train, y_train, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_train, y_train, verbose=0)[1],4)}")
print("--------------------------------------------")
print(f"Test Dataset:\n RPS: {np.round(model.evaluate(X_test, y_test, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_test, y_test, verbose=0)[1],4)}")
print("--------------------------------------------")

--------------------------------------------
Train Dataset:
 RPS: 0.1882, Accuracy: 0.5522
--------------------------------------------
Test Dataset:
 RPS: 0.1922, Accuracy: 0.5375
--------------------------------------------


In [950]:
y_pred = model.predict(X_test)

In [951]:
test_df = data2.iloc[data[data["season"] == 18].index]
test_df.reset_index(drop=True, inplace=True)
#clf_pred = pd.DataFrame(prediction, columns = ["A_pred_Clf", "D_pred_Clf", "H_pred_Clf"])
ann_pred = pd.DataFrame(y_pred, columns = ["H_pred_Ann", "D_pred_Ann", "A_pred_Ann"])
test_df2 = pd.concat([test_df, ann_pred], axis=1)
#test_df2["rps_Clf"] = rps(test_df2["H_pred_Clf"], test_df2["D_pred_Clf"], test_df2["A_pred_Clf"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])
test_df2["rps_ANN"] = rps(test_df2["H_pred_Ann"], test_df2["D_pred_Ann"], test_df2["A_pred_Ann"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])

In [952]:
print("--------------------------------------------")
print(f"Bookmaker RPS: {np.round(test_df2['rps_Book'].mean(), 4)}")
print("--------------------------------------------")
print(f"Neural Network RPS: {np.round(test_df2['rps_ANN'].mean(), 4)}")
print("--------------------------------------------")

--------------------------------------------
Bookmaker RPS: 0.1904
--------------------------------------------
Neural Network RPS: 0.1922
--------------------------------------------


In [953]:
a = test_df2[(test_df2["H_pred_Ann"] / test_df2["H_prob_odds"] -1 > 0.1)]["H_PL"].describe()
print(f"\nBets on Home Teams in 1X2 market:\n\nNr of Matches: {test_df2.shape[0]}\nNr of Bets: {int(a['count'])}\nAvg. Yield: {np.round(a['mean'] * 100, 2)}%\nMax winning odds: {np.round(a['max'] + 1, 2)}")
print("======================================================================")
b = test_df2[(test_df2["H_pred_Ann"] / test_df2["H_prob_odds"] -1 > 0.1)]["H_Ahc_PL"].describe()
print(f"\nBets on Home Teams in asian handicap market:\n\nNr of Matches: {test_df2.shape[0]}\nNr of Bets: {int(b['count'])}\nAvg. Yield: {np.round(b['mean'] * 100, 2)}%\nMax winning odds: {np.round(b['max'] + 1, 2)}")


Bets on Home Teams in 1X2 market:

Nr of Matches: 1332
Nr of Bets: 409
Avg. Yield: 3.79%
Max winning odds: 11.68

Bets on Home Teams in asian handicap market:

Nr of Matches: 1332
Nr of Bets: 409
Avg. Yield: -6.71%
Max winning odds: 2.36


In [954]:
c = test_df2[(test_df2["A_pred_Ann"] / test_df2["A_prob_odds"] -1 > 0.1)]["A_PL"].describe()
print(f"\nBets on Away Teams in 1X2 market:\n\nNr of Matches: {test_df2.shape[0]}\nNr of Bets: {int(c['count'])}\nAvg. Yield: {np.round(c['mean'] * 100, 2)}%\nMax winning odds: {np.round(c['max'] + 1, 2)}")
print("======================================================================")
d = test_df2[(test_df2["A_pred_Ann"] / test_df2["A_prob_odds"] -1 > 0.1)]["A_Ahc_PL"].describe()
print(f"\nBets on Away Teams in asian handicap market:\n\nNr of Matches: {test_df2.shape[0]}\nNr of Bets: {int(d['count'])}\nAvg. Yield: {np.round(d['mean'] * 100, 2)}%\nMax winning odds: {np.round(d['max'] + 1, 2)}")


Bets on Away Teams in 1X2 market:

Nr of Matches: 1332
Nr of Bets: 229
Avg. Yield: 7.69%
Max winning odds: 14.25

Bets on Away Teams in asian handicap market:

Nr of Matches: 1332
Nr of Bets: 229
Avg. Yield: 2.23%
Max winning odds: 2.26


In [955]:
e = test_df2[(test_df2["D_pred_Ann"] / test_df2["D_prob_odds"] -1 > 0.1)]["D_PL"].describe()
print(f"\nBets on Draws in 1X2 market:\n\nNr of Matches: {test_df2.shape[0]}\nNr of Bets: {int(e['count'])}\nAvg. Yield: {np.round(e['mean'] * 100, 2)}%\nMax winning odds: {np.round(e['max'] + 1, 2)}")


Bets on Draws in 1X2 market:

Nr of Matches: 1332
Nr of Bets: 318
Avg. Yield: 8.01%
Max winning odds: 5.65


In [922]:
model.save('clf_spa_new.h5')

In [298]:
model2 = load_model('clf_eng.h5', custom_objects={'loss_rps': loss_rps})

In [299]:
y_pred2 = model2.predict(X_test)

In [300]:
y_pred == y_pred2

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

### Reg

In [58]:
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train = df[df["season"] < season].iloc[:,:2]
    y_test = df[df["season"] == season].iloc[:,:2]
    return X_train, X_test, y_train, y_test

def loss_rps(y_true, y_pred):
    prob_h = y_pred[:, 0]
    prob_d = y_pred[:, 1]
    home = y_true[:, 0]
    draw = y_true[:, 1]

    step1 = prob_h - home
    step2 = prob_d - draw
    summe = step1 + step2
    return (step1 ** 2 + summe ** 2) / 2


data2 = df1[df1["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train, y_test = get_season(data, 18)

del X_train["season"]
del X_test["season"]

scaler = joblib.load("scaler_eng.save")

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [956]:
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'E', 'F', 'G', 'I', 'S', 'H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train = df[df["season"] < season].iloc[:,:2]
    y_test = df[df["season"] == season].iloc[:,:2]
    return X_train, X_test, y_train, y_test

def loss_rps(y_true, y_pred):
    prob_h = y_pred[:, 0]
    prob_d = y_pred[:, 1]
    home = y_true[:, 0]
    draw = y_true[:, 1]

    step1 = prob_h - home
    step2 = prob_d - draw
    summe = step1 + step2
    return (step1 ** 2 + summe ** 2) / 2


data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train, y_test = get_season(data, 18)

del X_train["season"]
del X_test["season"]

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1015]:
K.clear_session()

model = Sequential()
model.add(Dense(2048, kernel_initializer="he_normal", input_shape=(24,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(1024, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(256, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_initializer="he_normal"))
model.add(Activation('linear'))

model.compile(loss="logcosh", optimizer='adam', metrics=['accuracy'])

In [59]:
K.clear_session()

model = Sequential()
model.add(Dense(32, kernel_initializer="he_normal", input_shape=(19,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_initializer="he_normal"))
model.add(Activation('linear'))

model.compile(loss="logcosh", optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Train on 2798 samples, validate on 280 samples
Epoch 1/100
2798/2798 [==============================] - 1s 434us/step - loss: 0.9822 - acc: 0.5558 - val_loss: 0.6753 - val_acc: 0.7179
Epoch 2/100
2798/2798 [==============================] - 0s 59us/step - loss: 0.7822 - acc: 0.5722 - val_loss: 0.5855 - val_acc: 0.7429
Epoch 3/100
2798/2798 [==============================] - 0s 54us/step - loss: 0.6725 - acc: 0.5922 - val_loss: 0.5230 - val_acc: 0.7107
Epoch 4/100
2798/2798 [==============================] - 0s 59us/step - loss: 0.6186 - acc: 0.6094 - val_loss: 0.5032 - val_acc: 0.7214
Epoch 5/100
2798/2798 [==============================] - 0s 56us/step - loss: 0.5859 - acc: 0.6233 - val_loss: 0.4874 - val_acc: 0.7536
Epoch 6/100
2798/2798 [==============================] - 0s 66us/step - loss: 0.5593 - acc: 0.6623 - val_loss: 0.4746 - val_acc: 0.7464
Epoch 7/100
2798/2798 [==============================] - 0s 58us/step - loss: 0.5420 - acc: 0.6633 - val_loss: 0.4658 - val_acc: 0.7536


In [61]:
print(f"Train Dataset:\n RPS: {np.round(model.evaluate(X_train, y_train, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_train, y_train, verbose=0)[1],4)}")
print("--------------------------------------------")
print(f"Test Dataset:\n RPS: {np.round(model.evaluate(X_test, y_test, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_test, y_test, verbose=0)[1],4)}")
print("--------------------------------------------")

Train Dataset:
 RPS: 0.4479, Accuracy: 0.7202
--------------------------------------------
Test Dataset:
 RPS: 0.4428, Accuracy: 0.725
--------------------------------------------


In [62]:
y_pred = model.predict(X_test)

In [63]:
test_df = data2.iloc[data[data["season"] == 18].index]
test_df.reset_index(drop=True, inplace=True)
#clf_pred = pd.DataFrame(prediction, columns = ["A_pred_Clf", "D_pred_Clf", "H_pred_Clf"])
ann_pred = pd.DataFrame(y_pred, columns = ["H_pred_Goals", "A_pred_Goals"])
test_df2 = pd.concat([test_df, ann_pred], axis=1)
#test_df2["rps_Clf"] = rps(test_df2["H_pred_Clf"], test_df2["D_pred_Clf"], test_df2["A_pred_Clf"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])
#test_df2["rps_ANN"] = rps(test_df2["H_pred_Ann"], test_df2["D_pred_Ann"], test_df2["A_pred_Ann"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])

In [64]:
test_df2.columns

Index(['AC', 'AF', 'AR', 'AS', 'AST', 'AY', 'A_Ahc_PL', 'A_Def_Rat',
       'A_Form_Tot4', 'A_Off_Rat',
       ...
       'WHH', 'round', 'rps_Book', 'rps_Mas', 'rps_PoiMas', 'rps_Poi_mix',
       'season', 'xGD_Mas', 'H_pred_Goals', 'A_pred_Goals'],
      dtype='object', length=180)

In [65]:
test_df2["ANN_pred_hc"] = test_df2["A_pred_Goals"] - test_df2["H_pred_Goals"]

In [66]:
test_df2["Hc_Diff"] = test_df2["BbAHh"] - test_df2["ANN_pred_hc"]

In [85]:
test_df2[test_df2["Hc_Diff"] > 0.05].H_Ahc_PL.describe(), test_df2[test_df2["Hc_Diff"] > 0.05].H_PL.describe()

(count    129.000000
 mean       0.006783
 std        0.836155
 min       -1.000000
 25%       -1.000000
 50%        0.325000
 75%        0.830000
 max        1.170000
 Name: H_Ahc_PL, dtype: float64, count    129.000000
 mean       0.035349
 std        1.639838
 min       -1.000000
 25%       -1.000000
 50%       -1.000000
 75%        0.810000
 max       10.680000
 Name: H_PL, dtype: float64)

In [86]:
test_df2[test_df2["Hc_Diff"] < -0.05].A_Ahc_PL.describe(), test_df2[test_df2["Hc_Diff"] < -0.05].A_PL.describe()

(count    114.000000
 mean       0.026316
 std        0.835636
 min       -1.000000
 25%       -1.000000
 50%        0.382500
 75%        0.810000
 max        1.140000
 Name: A_Ahc_PL, dtype: float64, count    114.000000
 mean      -0.039386
 std        2.252422
 min       -1.000000
 25%       -1.000000
 50%       -1.000000
 75%        0.515000
 max       17.000000
 Name: A_PL, dtype: float64)

In [1012]:
model.save('reg_efgis.h5')

In [39]:
model3 = load_model('reg_spa.h5')

In [40]:
y_pred3 = model3.predict(X_test)

In [972]:
test_df2[test_df2["Hc_Diff"] > 0.1].groupby("round")["H_Ahc_PL"].sum()#[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "H_pred_Goals", "A_pred_Goals"]]

round
11    -3.290
12     1.300
13     2.220
14     3.195
15    -5.100
16     1.490
17    -0.355
18     3.445
19    -1.785
20    -2.615
21    -3.730
22     3.540
23     0.355
24    -7.280
25    -1.480
26     3.365
27    10.060
28    -3.135
29     3.040
30    -3.790
31    -0.560
32    -1.570
33     0.190
34    -1.535
35     2.230
36    -3.475
37     4.260
38     1.855
Name: H_Ahc_PL, dtype: float64

In [973]:
test_df2[test_df2["Hc_Diff"] < -0.1].groupby("round")["A_Ahc_PL"].sum()#[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "H_pred_Goals", "A_pred_Goals"]]

round
11    2.570
12    0.300
13    5.790
14    3.150
15    2.720
16   -0.650
17    4.595
18    6.550
19   -0.390
20    0.000
21    0.260
22    1.630
23   -1.460
24   -1.020
25    1.010
26   -2.770
27    0.165
28    1.475
29   -1.065
30   -0.175
31   -1.255
32   -1.600
33    0.110
34    4.010
35   -0.195
36    0.165
37    5.295
38    2.225
Name: A_Ahc_PL, dtype: float64

In [47]:
test_df2.columns

Index(['AC', 'AF', 'AR', 'AS', 'AST', 'AY', 'A_Ahc_PL', 'A_Def_Rat',
       'A_Form_Tot4', 'A_Off_Rat',
       ...
       'rps_Book', 'rps_Mas', 'rps_PoiMas', 'rps_Poi_mix', 'season', 'xGD_Mas',
       'H_pred_Goals', 'A_pred_Goals', 'ANN_pred_hc', 'Hc_Diff'],
      dtype='object', length=182)

In [992]:
test_df2[test_df2["Hc_Diff"] > 0.05].groupby("")["H_Ahc_PL"].sum()

E
0   -3.840
1    9.055
Name: H_Ahc_PL, dtype: float64

In [989]:
test_df2[test_df2["Hc_Diff"] < -0.05]["A_Ahc_PL"].sum()

42.95499999999999

### XGBoost test

In [87]:
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgS', 'A_avgS', 'H_avgS_c', 'A_avgS_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'GoalDiff_last_away', 'GoalDiff_last_home', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4', 'H_prob_odds', 'D_prob_odds', 'A_prob_odds']

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train_hg = df[df["season"] < season].iloc[:,0]
    y_test_hg = df[df["season"] == season].iloc[:,0]
    y_train_ag = df[df["season"] < season].iloc[:,1]
    y_test_ag = df[df["season"] == season].iloc[:,1]
    return X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag

data2 = df1[df1["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag = get_season(data, 18)

del X_train["season"]
del X_test["season"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
#home
model_h = XGBRegressor(n_estimators=50, max_depth=4)
model_h.fit(X_train, y_train_hg)

#away
model_a = XGBRegressor(n_estimators=50, max_depth=4)
model_a.fit(X_train, y_train_ag)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [90]:
home = model_h.predict(X_test)
away = model_a.predict(X_test)

In [91]:
test_df = data2.iloc[data[data["season"] == 18].index]
test_df.reset_index(drop=True, inplace=True)
#ann_pred = pd.DataFrame(y_pred, columns = ["H_pred_Goals", "A_pred_Goals"])
test_df["H_pred_Goals"] = home
test_df["A_pred_Goals"] = away
#test_df2["rps_Clf"] = rps(test_df2["H_pred_Clf"], test_df2["D_pred_Clf"], test_df2["A_pred_Clf"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])
#test_df2["rps_ANN"] = rps(test_df2["H_pred_Ann"], test_df2["D_pred_Ann"], test_df2["A_pred_Ann"], test_df2["Home"], test_df2["Draw"], test_df2["Away"])

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [92]:
test_df["XGB_pred_hc"] = test_df["A_pred_Goals"] - test_df["H_pred_Goals"]
test_df["Hc_Diff"] = test_df["BbAHh"] - test_df["XGB_pred_hc"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [105]:
test_df[(test_df["Hc_Diff"] > 0.1) & (test_df["BbAvAHH"] > 1.9)].H_Ahc_PL.describe(), test_df[test_df["Hc_Diff"] > 0.5].H_PL.describe()

(count    58.000000
 mean      0.031810
 std       0.888714
 min      -1.000000
 25%      -1.000000
 50%       0.000000
 75%       0.940000
 max       1.220000
 Name: H_Ahc_PL, dtype: float64, count    27.000000
 mean      0.527037
 std       1.676821
 min      -1.000000
 25%      -1.000000
 50%       0.440000
 75%       1.135000
 max       6.550000
 Name: H_PL, dtype: float64)

In [106]:
test_df[(test_df["Hc_Diff"] < -0.1) & (test_df["BbAvAHA"] > 1.9)].A_Ahc_PL.describe(), test_df[test_df["Hc_Diff"] < -0.5].A_PL.describe()

(count    44.000000
 mean     -0.156591
 std       0.858646
 min      -1.000000
 25%      -1.000000
 50%      -0.500000
 75%       0.635000
 max       1.150000
 Name: A_Ahc_PL, dtype: float64, count    17.000000
 mean     -0.503529
 std       0.861945
 min      -1.000000
 25%      -1.000000
 50%      -1.000000
 75%       0.240000
 max       1.850000
 Name: A_PL, dtype: float64)

In [33]:
test_df["A_Ahc_PL"].describe()

count    280.000000
mean      -0.066357
std        0.850441
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.810000
max        1.230000
Name: A_Ahc_PL, dtype: float64

In [108]:
1/test_df["BbAvAHH"] + 1/test_df["BbAvAHA"]

0      1.039091
1      1.028611
2      1.031283
3      1.033764
4      1.030245
5      1.039428
6      1.032593
7      1.033151
8      1.034759
9      1.033674
10     1.035731
11     1.032542
12     1.037988
13     1.034759
14     1.033599
15     1.034759
16     1.031366
17     1.034887
18     1.033255
19     1.035590
20     1.035310
21     1.036545
22     1.031885
23     1.034253
24     1.032272
25     1.037107
26     1.031915
27     1.033930
28     1.034759
29     1.036545
         ...   
250    1.037255
251    1.038968
252    1.034759
253    1.042119
254    1.033151
255    1.035578
256    1.040111
257    1.033654
258    1.041280
259    1.035590
260    1.034427
261    1.036269
262    1.036269
263    1.036297
264    1.035731
265    1.039136
266    1.042225
267    1.039647
268    1.037272
269    1.039059
270    1.039136
271    1.039024
272    1.036297
273    1.039529
274    1.039428
275    1.039059
276    1.036381
277    1.036965
278    1.037634
279    1.036089
Length: 280, dtype: floa

In [2]:
import pandas as pd

In [22]:
eng = pd.read_csv("England_all_16.csv")
fra = pd.read_csv("France_all_16.csv")
ita = pd.read_csv("Italy_all_16.csv")
ger = pd.read_csv("Germany_all_16.csv")
spa = pd.read_csv("Spain_all_16.csv")

data = pd.concat([eng, fra, ita, ger, spa])
data.reset_index(drop=True, inplace=True)

In [23]:
data[["rps_Book", "rps_Rdf", "rps_Xgb", "rps_ANN", "rps_avg"]].describe()

,rps_Book,rps_Rdf,rps_Xgb,rps_ANN,rps_avg
count,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000
mean,0.196054,0.198840,0.201671,0.196002,0.197318
std,0.134795,0.130556,0.144134,0.134966,0.133658
min,0.002399,0.003040,0.001193,0.006165,0.003648
25%,0.116891,0.120541,0.104724,0.119066,0.115173
50%,0.159694,0.165702,0.161436,0.162614,0.163399
75%,0.253820,0.254590,0.262079,0.253647,0.251659
max,0.843500,0.866656,0.803792,0.843625,0.835653


In [24]:
value = 0.1

data[data["H_pred_avg"] / data["H_prob_odds"] - 1 > value].H_Ahc_PL.describe()

count    247.000000
mean       0.019717
std        0.882859
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.890000
max        1.350000
Name: H_Ahc_PL, dtype: float64

In [25]:
data[data["A_pred_avg"] / data["A_prob_odds"] - 1 > value].A_Ahc_PL.describe()

count    270.000000
mean      -0.033296
std        0.866043
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.840000
max        1.220000
Name: A_Ahc_PL, dtype: float64

In [33]:
difference = 0.2

data[data["Hc_Diff_avg"] > difference].H_Ahc_PL.describe()

count    256.000000
mean       0.085391
std        0.840537
min       -1.000000
25%       -1.000000
50%        0.490000
75%        0.850000
max        1.180000
Name: H_Ahc_PL, dtype: float64

In [27]:
data[data["Hc_Diff_avg"] < -difference].A_Ahc_PL.describe()

count    375.000000
mean       0.018480
std        0.866511
min       -1.000000
25%       -1.000000
50%        0.340000
75%        0.840000
max        1.180000
Name: A_Ahc_PL, dtype: float64

In [36]:
difference = 0.05
value = 0.05

data[(data["Hc_Diff_avg"] > difference) & (data["H_pred_avg"] / data["H_prob_odds"] - 1 > value)].H_Ahc_PL.describe()

count    237.000000
mean      -0.018333
std        0.848843
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.840000
max        1.350000
Name: H_Ahc_PL, dtype: float64

In [29]:
data[(data["Hc_Diff_avg"] < -difference) & (data["A_pred_avg"] / data["A_prob_odds"] - 1 > value)].A_Ahc_PL.describe()

count    270.000000
mean       0.024593
std        0.873342
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.860000
max        1.220000
Name: A_Ahc_PL, dtype: float64

In [11]:
for i in [0, 0.05, 0.1, 0.15, 0.2, 0.25]:
    for j in [0, 0.05, 0.1, 0.15, 0.2, 0.25]:
        h = data[(data["Hc_Diff_avg"] > i) & (data["H_pred_avg"] / data["H_prob_odds"] - 1 > j)].H_Ahc_PL.describe()
        a = data[(data["Hc_Diff_avg"] < -i) & (data["A_pred_avg"] / data["A_prob_odds"] - 1 > j)].A_Ahc_PL.describe()
        print(f"-----diff = {i}-----value = {j}-------")
        print(f"Home: {h['mean']} , {h['count']}")
        print(f"Away: {a['mean']} , {a['count']}")
        print(f"Total: {(h['mean']*h['count']+a['mean']*a['count'])/(h['count']+a['count'])} , {h['count']+a['count']}")

-----diff = 0-----value = 0-------
Home: 0.004555808656036452 , 439.0
Away: 0.009262048192771077 , 332.0
Total: 0.006582360570687419 , 771.0
-----diff = 0-----value = 0.05-------
Home: -0.017292307692307694 , 325.0
Away: 0.06381632653061221 , 245.0
Total: 0.01757017543859648 , 570.0
-----diff = 0-----value = 0.1-------
Home: -0.01673728813559322 , 236.0
Away: 0.1279234972677596 , 183.0
Total: 0.04644391408114559 , 419.0
-----diff = 0-----value = 0.15-------
Home: -0.018898809523809512 , 168.0
Away: 0.1695833333333333 , 132.0
Total: 0.06403333333333333 , 300.0
-----diff = 0-----value = 0.2-------
Home: -0.06965811965811965 , 117.0
Away: 0.16451086956521732 , 92.0
Total: 0.03342105263157893 , 209.0
-----diff = 0-----value = 0.25-------
Home: -0.07852272727272726 , 88.0
Away: 0.14171428571428563 , 70.0
Total: 0.01905063291139238 , 158.0
-----diff = 0.05-----value = 0-------
Home: 0.010346666666666683 , 375.0
Away: 0.04861111111111109 , 306.0
Total: 0.027540381791483118 , 681.0
-----diff =

In [35]:
df1[(df1["A_Form_Tot4"] >= 1.25) & (df1["H_prob_odds"] >= 0.5)].groupby("season").H_Ahc_PL.describe()

,count,mean,std,min,25%,50%,75%,max
season,,,,,,,,
8,27.0,0.302037,0.758453,-1.0,-0.25,0.7800,0.86000,1.06
9,37.0,0.066892,0.793321,-1.0,-1.00,0.2000,0.79000,1.07
10,33.0,0.155455,0.545430,-1.0,0.03,0.1000,0.32000,1.00
11,36.0,-0.138194,0.829570,-1.0,-1.00,0.0000,0.77250,1.12
12,37.0,-0.004865,0.931825,-1.0,-1.00,0.0000,0.91000,1.11
13,31.0,-0.115645,0.868415,-1.0,-1.00,0.0000,0.77500,1.04
14,36.0,-0.006667,0.911605,-1.0,-1.00,0.0000,0.87000,1.20
15,35.0,0.218429,0.913317,-1.0,-1.00,0.7500,0.97500,1.22
16,42.0,0.003333,0.950898,-1.0,-1.00,0.2325,0.92500,1.22


In [32]:
df1[df1["H_Form_Tot4"] >= 1.2].A_Ahc_PL.describe()

count    1156.000000
mean       -0.049079
std         0.914060
min        -1.000000
25%        -1.000000
50%         0.000000
75%         0.860000
max         7.490000
Name: A_Ahc_PL, dtype: float64